### Current Process
1. Read in data --> Done

2. Custom Imputation --> Done

3. Add Binary Class --> Done, Should Add Binary Class Later

4. Summary Statistics Features --> Done

5. Wrapper Functions --> Done, Need to Test Though

6. Sklearn Pipeline Categorical Features --> One Hot Encoding Done

7. Sklearn Pipeline Numerical Features --> StandardScaler Done

8. Create Lagged Features --> Done

9. Modeling --> Currently XgBoost, (Maybe Try: TensorFlow Decision Tree, TensorFlow Probability Model)

10. Model Evaluation --> Accuracy, Precision, Recall, F1, Confusion Matrix (Need to add Variable Importance Based on Variance)

11. PySpark: XGBoost Classification Feature Importance

In [ ]:
# # Need to Run These in Notebook Version For Pandas UDF
! pip install pyarrow
! pip install pandas
! pip install scikit-learn
! pip install pyspark
! pip install xgboost
! pip install kaleido
! pip install EntropyHub

In [1]:
from Input_Variables.read_vars import train_data_storage, validation_data_storage, test_data_storage, \
                                      one_hot_encoding_data, \
                                      analysis_group, \
                                      daily_stats_features_lower, daily_stats_features_upper, \
                                      model_storage_location, random_seed, \
                                      time_series_lag_values_created, \
                                      evaluation_metrics_output_storage, \
                                      feature_importance_storage_location, \
                                      overall_feature_importance_plot_location

from Data_Schema.schema import Pandas_UDF_Data_Schema
from Read_In_Data.read_data import Reading_Data
from Data_Pipeline.imputation_pipeline import Date_And_Value_Imputation


from Feature_Generation.create_binary_labels import Create_Binary_Labels
from Feature_Generation.summary_stats import Summary_Stats_Features
from Feature_Generation.lag_features import Create_Lagged_Features
from Feature_Generation.time_series_feature_creation import TS_Features
from Feature_Generation.difference_features import Difference_Features

from Data_Pipeline.encoding_scaling_pipeline import Feature_Transformations

from Model_Creation.pyspark_xgboost import Create_PySpark_XGBoost

from Model_Predictions.pyspark_model_preds import Model_Predictions

from Model_Evaluation.pyspark_model_eval import Evaluate_Model

from Feature_Importance.model_feature_importance import Feature_Importance

from Model_Plots.xgboost_classification_plots import XGBoost_Classification_Plot

# General Modules

In [2]:
# PySpark UDF Schema Activation
pandas_udf_data_schema=Pandas_UDF_Data_Schema()

# Data Location
reading_data=Reading_Data()

# Create Binary y Variables
create_binary_labels=Create_Binary_Labels()

# Imputation
date_and_value_imputation=Date_And_Value_Imputation()

# Features Daily Stats Module
summary_stats_features=Summary_Stats_Features()

# Features Complex
ts_features=TS_Features()

# Features Lagged Value
create_lag_features=Create_Lagged_Features()

# Features Differences
difference_features=Difference_Features()

# PySpark XGBoost Model Module
create_pyspark_xgboost=Create_PySpark_XGBoost()

# Classification Evaluation
evaluate_model=Evaluate_Model()

# Model Plots Feature Importance
xgboost_classification_plot=XGBoost_Classification_Plot()

# Feature Transformations
feature_transformations=Feature_Transformations()


pyspark_custom_imputation_schema=pandas_udf_data_schema.custom_imputation_pyspark_schema()


model_predictions=Model_Predictions()

# Feature Importance
feature_importance=Feature_Importance()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/04 19:41:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# PySpark

### 1. PySpark: Reading In Data

#### Training

In [3]:
training_df=reading_data.read_in_pyspark_training(training_data_location=train_data_storage)
training_df.show()

+--------------------+-----+-------------------+---------------------+------------------+
|           PatientId|Value| GlucoseDisplayTime|GlucoseDisplayTimeRaw|GlucoseDisplayDate|
+--------------------+-----+-------------------+---------------------+------------------+
|8W/rpnb48OMm47W2x...|328.0|2022-01-31 17:38:00| 2022-01-31T17:38:...|        2022-01-31|
|8W/rpnb48OMm47W2x...|331.0|2022-01-31 17:43:00| 2022-01-31T17:43:...|        2022-01-31|
|8W/rpnb48OMm47W2x...|329.0|2022-01-31 17:48:00| 2022-01-31T17:48:...|        2022-01-31|
|8W/rpnb48OMm47W2x...|321.0|2022-01-31 17:53:00| 2022-01-31T17:53:...|        2022-01-31|
|8W/rpnb48OMm47W2x...|315.0|2022-01-31 17:58:00| 2022-01-31T17:58:...|        2022-01-31|
|8W/rpnb48OMm47W2x...|313.0|2022-01-31 18:03:00| 2022-01-31T18:03:...|        2022-01-31|
|8W/rpnb48OMm47W2x...|304.0|2022-01-31 18:08:00| 2022-01-31T18:08:...|        2022-01-31|
|8W/rpnb48OMm47W2x...|298.0|2022-01-31 18:13:00| 2022-01-31T18:13:...|        2022-01-31|
|8W/rpnb48

#### Testing

In [4]:
testing_df=reading_data.read_in_pyspark_testing(testing_data_location=test_data_storage)
testing_df.show()

+--------------------+-----+-------------------+---------------------+------------------+
|           PatientId|Value| GlucoseDisplayTime|GlucoseDisplayTimeRaw|GlucoseDisplayDate|
+--------------------+-----+-------------------+---------------------+------------------+
|8W/rpnb48OMm47W2x...|  0.0|2022-02-08 16:59:00| 2022-02-08T16:59:...|        2022-02-08|
|8W/rpnb48OMm47W2x...|  0.0|2022-02-08 17:04:00| 2022-02-08T17:04:...|        2022-02-08|
|8W/rpnb48OMm47W2x...|  0.0|2022-02-08 17:09:00| 2022-02-08T17:09:...|        2022-02-08|
|8W/rpnb48OMm47W2x...|  0.0|2022-02-08 17:14:00| 2022-02-08T17:14:...|        2022-02-08|
|8W/rpnb48OMm47W2x...|  0.0|2022-02-08 17:19:00| 2022-02-08T17:19:...|        2022-02-08|
|8W/rpnb48OMm47W2x...|  0.0|2022-02-08 17:24:00| 2022-02-08T17:24:...|        2022-02-08|
|8W/rpnb48OMm47W2x...|277.0|2022-02-08 17:29:00| 2022-02-08T17:29:...|        2022-02-08|
|8W/rpnb48OMm47W2x...|270.0|2022-02-08 17:34:00| 2022-02-08T17:34:...|        2022-02-08|
|8W/rpnb48

### 2. PySpark: Custom Imputation Pipeline

#### Training

In [5]:
training_custom_imputation_schema=pandas_udf_data_schema.custom_imputation_pyspark_schema()
training_custom_imputation_pipeline=date_and_value_imputation.\
                                        pyspark_custom_imputation_pipeline(df=training_df, 
                                                                           output_schema=pyspark_custom_imputation_schema,
                                                                           analysis_group=analysis_group)

training_custom_imputation_pipeline.show(1)

/home/jovyan/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/pyspark/sql/pandas/group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+-------------------+--------------------+-----+
| GlucoseDisplayTime|           PatientId|Value|
+-------------------+--------------------+-----+
|2022-01-31 17:35:00|8W/rpnb48OMm47W2x...|328.0|
+-------------------+--------------------+-----+
only showing top 1 row



#### Testing

In [6]:
testing_custom_imputation_schema=pandas_udf_data_schema.custom_imputation_pyspark_schema()
testing_custom_imputation_pipeline=date_and_value_imputation.\
                                        pyspark_custom_imputation_pipeline(df=testing_df, 
                                                                           output_schema=pyspark_custom_imputation_schema,
                                                                           analysis_group=analysis_group)

testing_custom_imputation_pipeline.show(1)

+-------------------+--------------------+---------+
| GlucoseDisplayTime|           PatientId|    Value|
+-------------------+--------------------+---------+
|2022-02-08 16:55:00|8W/rpnb48OMm47W2x...|164.20488|
+-------------------+--------------------+---------+
only showing top 1 row



### 3. PySpark: Adding Binary Labels

#### Training

In [7]:
training_df_added_binary_labels=create_binary_labels.pyspark_binary_labels(df=training_custom_imputation_pipeline)

training_df_added_binary_labels.show(1)

+-------------------+--------------------+-----+--------+--------+--------+
| GlucoseDisplayTime|           PatientId|Value|y_Binary|is_above|is_below|
+-------------------+--------------------+-----+--------+--------+--------+
|2022-01-31 17:35:00|8W/rpnb48OMm47W2x...|328.0|       1|       1|       0|
+-------------------+--------------------+-----+--------+--------+--------+
only showing top 1 row



#### Testing

In [8]:
testing_df_added_binary_labels=create_binary_labels.pyspark_binary_labels(df=testing_custom_imputation_pipeline)

testing_df_added_binary_labels.show(1, truncate=False)

+-------------------+--------------------------------------------+---------+--------+--------+--------+
|GlucoseDisplayTime |PatientId                                   |Value    |y_Binary|is_above|is_below|
+-------------------+--------------------------------------------+---------+--------+--------+--------+
|2022-02-08 16:55:00|8W/rpnb48OMm47W2x4FSkc7+9u2mol061DQuJoMdiK0=|164.20488|0       |0       |0       |
+-------------------+--------------------------------------------+---------+--------+--------+--------+
only showing top 1 row



### 4. PySpark: Feature Creation

#### Training

##### Complex Features

In [9]:
training_df_differences = difference_features.add_difference_features(training_df_added_binary_labels)
training_df_differences.show(5)

+-------------------+--------------------+-----+--------+--------+--------+---------+-------+
| GlucoseDisplayTime|           PatientId|Value|y_Binary|is_above|is_below|FirstDiff|SecDiff|
+-------------------+--------------------+-----+--------+--------+--------+---------+-------+
|2022-01-31 17:35:00|8W/rpnb48OMm47W2x...|328.0|       1|       1|       0|      0.0|    0.0|
|2022-01-31 17:40:00|8W/rpnb48OMm47W2x...|331.0|       1|       1|       0|      3.0|    3.0|
|2022-01-31 17:45:00|8W/rpnb48OMm47W2x...|329.0|       1|       1|       0|     -2.0|   -5.0|
|2022-01-31 17:50:00|8W/rpnb48OMm47W2x...|321.0|       1|       1|       0|     -8.0|   -6.0|
|2022-01-31 17:55:00|8W/rpnb48OMm47W2x...|315.0|       1|       1|       0|     -6.0|    2.0|
+-------------------+--------------------+-----+--------+--------+--------+---------+-------+
only showing top 5 rows



In [10]:
training_df_chunks = summary_stats_features.create_chunk_col(training_df_differences, chunk_val = 288)
training_df_chunks.show(5)

+-------------------+--------------------+-----+--------+--------+--------+---------+-------+-----+-----+
| GlucoseDisplayTime|           PatientId|Value|y_Binary|is_above|is_below|FirstDiff|SecDiff|index|Chunk|
+-------------------+--------------------+-----+--------+--------+--------+---------+-------+-----+-----+
|2022-01-31 17:35:00|8W/rpnb48OMm47W2x...|328.0|       1|       1|       0|      0.0|    0.0|    1|    0|
|2022-01-31 17:40:00|8W/rpnb48OMm47W2x...|331.0|       1|       1|       0|      3.0|    3.0|    2|    0|
|2022-01-31 17:45:00|8W/rpnb48OMm47W2x...|329.0|       1|       1|       0|     -2.0|   -5.0|    3|    0|
|2022-01-31 17:50:00|8W/rpnb48OMm47W2x...|321.0|       1|       1|       0|     -8.0|   -6.0|    4|    0|
|2022-01-31 17:55:00|8W/rpnb48OMm47W2x...|315.0|       1|       1|       0|     -6.0|    2.0|    5|    0|
+-------------------+--------------------+-----+--------+--------+--------+---------+-------+-----+-----+
only showing top 5 rows



In [11]:
# training_df_poincare = training_df_chunks.groupby(['PatientId', 'Chunk']).apply(ts_features.poincare)
# training_df_poincare.show(5)

# training_df_entropy = training_df_chunks.groupby(['PatientId', 'Chunk']).apply(ts_features.entropy)
# training_df_entropy.show(5)

In [12]:
# training_df_complex_features = training_df_poincare.join(training_df_entropy,['PatientId', 'Chunk'])
# training_df_complex_features.show()

In [13]:
# training_df_sleep = ts_features.process_for_sleep(df=training_df_added_binary_labels)
# training_df_sleep.show(5)

##### Statistical Features

In [14]:
training_features_summary_stats=summary_stats_features.pyspark_summary_statistics(df=training_df_chunks)
# merge complex features and summary stats and demographics and sleep features
# merge in one hot encoded cohort file info demographics
    # '/cephfs/data/cohort_encoded.parquet' (gender, treatment, age category)
    # groupby patientId and chunk

training_features_summary_stats.show(3)

+--------------------+-----+------------------+------------------+------+-----+-----+--------------------+--------------------+------------------+------------------+----------+----------+---------------+------+
|           PatientId|Chunk|              Mean|            StdDev|Median|  Min|  Max|        AvgFirstDiff|          AvgSecDiff|      StdFirstDiff|        StdSecDiff|CountAbove|CountBelow|TotalOutOfRange|target|
+--------------------+-----+------------------+------------------+------+-----+-----+--------------------+--------------------+------------------+------------------+----------+----------+---------------+------+
|8W/rpnb48OMm47W2x...|    1|250.30549891789755| 60.39017986635174| 240.0|161.0|401.0|-0.10877741707695855|0.003472222222222222|5.9481085344362254| 5.444201337481776|       134|         0|            134|  -105|
|8W/rpnb48OMm47W2x...|    2|214.40972222222223|42.174007087187746| 207.0|160.0|301.0|  0.2673611111111111|-0.00694444444444...| 4.337830184821668|3.80719386

In [15]:
# #add target variable
# training_features_final_summary = summary_stats_features\
#                                     .add_lag_out_of_range(df=training_features_summary_stats, chunk_lag=1)

# training_features_final_summary.show(4)

In [16]:
# Merge these together
# training_features_summary_stats
# training_df_complex_features
# one-hot-encoding 

#### Testing

##### Complex Features

In [17]:
testing_df_differences = difference_features.add_difference_features(testing_df_added_binary_labels)
testing_df_differences.show(5)

+-------------------+--------------------+---------+--------+--------+--------+---------+-------+
| GlucoseDisplayTime|           PatientId|    Value|y_Binary|is_above|is_below|FirstDiff|SecDiff|
+-------------------+--------------------+---------+--------+--------+--------+---------+-------+
|2022-02-08 16:55:00|8W/rpnb48OMm47W2x...|164.20488|       0|       0|       0|      0.0|    0.0|
|2022-02-08 17:00:00|8W/rpnb48OMm47W2x...|164.20488|       0|       0|       0|      0.0|    0.0|
|2022-02-08 17:05:00|8W/rpnb48OMm47W2x...|164.20488|       0|       0|       0|      0.0|    0.0|
|2022-02-08 17:10:00|8W/rpnb48OMm47W2x...|164.20488|       0|       0|       0|      0.0|    0.0|
|2022-02-08 17:15:00|8W/rpnb48OMm47W2x...|164.20488|       0|       0|       0|      0.0|    0.0|
+-------------------+--------------------+---------+--------+--------+--------+---------+-------+
only showing top 5 rows



In [18]:
testing_df_chunks = summary_stats_features.create_chunk_col(testing_df_differences, chunk_val = 288)
testing_df_chunks.show(5)

# testing_df_poincare = testing_df_chunks.groupby(['PatientId', 'Chunk']).apply(ts_features.poincare)
# testing_df_poincare.show(5)

# testing_df_entropy = testing_df_chunks.groupby(['PatientId', 'Chunk']).apply(ts_features.entropy)
# testing_df_entropy.show(5)

+-------------------+--------------------+---------+--------+--------+--------+---------+-------+-----+-----+
| GlucoseDisplayTime|           PatientId|    Value|y_Binary|is_above|is_below|FirstDiff|SecDiff|index|Chunk|
+-------------------+--------------------+---------+--------+--------+--------+---------+-------+-----+-----+
|2022-02-08 16:55:00|8W/rpnb48OMm47W2x...|164.20488|       0|       0|       0|      0.0|    0.0|    1|    0|
|2022-02-08 17:00:00|8W/rpnb48OMm47W2x...|164.20488|       0|       0|       0|      0.0|    0.0|    2|    0|
|2022-02-08 17:05:00|8W/rpnb48OMm47W2x...|164.20488|       0|       0|       0|      0.0|    0.0|    3|    0|
|2022-02-08 17:10:00|8W/rpnb48OMm47W2x...|164.20488|       0|       0|       0|      0.0|    0.0|    4|    0|
|2022-02-08 17:15:00|8W/rpnb48OMm47W2x...|164.20488|       0|       0|       0|      0.0|    0.0|    5|    0|
+-------------------+--------------------+---------+--------+--------+--------+---------+-------+-----+-----+
only showi

In [19]:
# testing_df_complex_features = testing_df_poincare.join(testing_df_entropy,['PatientId', 'Chunk'])
# testing_df_complex_features.show()

In [20]:
# training_df_sleep = ts_features.process_for_sleep(df=testing_df_added_binary_labels)
# training_df_sleep.show(5)

##### Statistical Features

In [21]:
testing_features_summary_stats=summary_stats_features.pyspark_summary_statistics(df=testing_df_chunks)

# merge complex features and summary stats and demographics and sleep features
# merge in one hot encoded cohort file info demographics
    # '/cephfs/data/cohort_encoded.parquet' (gender, treatment, age category)
    # groupby patientId and chunk

testing_features_summary_stats.show(3)

+--------------------+-----+------------------+------------------+------+-----+-----+-------------------+-------------------+------------------+------------------+----------+----------+---------------+------+
|           PatientId|Chunk|              Mean|            StdDev|Median|  Min|  Max|       AvgFirstDiff|         AvgSecDiff|      StdFirstDiff|        StdSecDiff|CountAbove|CountBelow|TotalOutOfRange|target|
+--------------------+-----+------------------+------------------+------+-----+-----+-------------------+-------------------+------------------+------------------+----------+----------+---------------+------+
|8W/rpnb48OMm47W2x...|    1| 139.6583505206638|36.561064568332455| 135.0| 66.0|247.0| 0.1701388888888889|             0.0625|  8.68154980903809|12.418971213537594|         3|         1|              4|    49|
|8W/rpnb48OMm47W2x...|    2|253.55555555555554| 5.246692079565728| 255.0|245.0|260.0|-0.2222222222222222|-0.8888888888888888|  3.73422608373468|2.6193722742502854| 

In [22]:
# Merge these together
# testing_features_summary_stats
# training_df_complex_features
# one-hot-encoding 

### 7. PySpark: Sklearn Categorical Pipeline in PySpark

In [23]:
one_hot_encoded_df=reading_data.read_in_one_hot_encoded_data(one_hot_encoding_location=one_hot_encoding_data)
one_hot_encoded_df=one_hot_encoded_df.select('UserId', 
                                             'Sex_Encoded', 
                                             'Treatment_Encoded', 
                                             'AgeGroup_Encoded')

#### Training

In [24]:
training_encoded=training_features_summary_stats.join(one_hot_encoded_df,
                                                       training_features_summary_stats.PatientId==one_hot_encoded_df.UserId,
                                                       "left")


In [25]:
training_encoded.show(4)

+--------------------+-----+------------------+------------------+------+-----+-----+--------------------+--------------------+------------------+------------------+----------+----------+---------------+------+--------------------+-----------+-----------------+----------------+
|           PatientId|Chunk|              Mean|            StdDev|Median|  Min|  Max|        AvgFirstDiff|          AvgSecDiff|      StdFirstDiff|        StdSecDiff|CountAbove|CountBelow|TotalOutOfRange|target|              UserId|Sex_Encoded|Treatment_Encoded|AgeGroup_Encoded|
+--------------------+-----+------------------+------------------+------+-----+-----+--------------------+--------------------+------------------+------------------+----------+----------+---------------+------+--------------------+-----------+-----------------+----------------+
|8W/rpnb48OMm47W2x...|    1|250.30549891789755| 60.39017986635174| 240.0|161.0|401.0|-0.10877741707695855|0.003472222222222222|5.9481085344362254| 5.44420133748177

In [26]:
# merge training_features_summary with 

#### Testing

In [27]:
testing_encoded=testing_features_summary_stats.join(one_hot_encoded_df,
                                                       testing_features_summary_stats.PatientId==one_hot_encoded_df.UserId,
                                                       "left")

In [28]:
testing_encoded.show(4)

+--------------------+-----+------------------+------------------+------+-----+-----+-------------------+--------------------+------------------+------------------+----------+----------+---------------+------+--------------------+-------------+-----------------+----------------+
|           PatientId|Chunk|              Mean|            StdDev|Median|  Min|  Max|       AvgFirstDiff|          AvgSecDiff|      StdFirstDiff|        StdSecDiff|CountAbove|CountBelow|TotalOutOfRange|target|              UserId|  Sex_Encoded|Treatment_Encoded|AgeGroup_Encoded|
+--------------------+-----+------------------+------------------+------+-----+-----+-------------------+--------------------+------------------+------------------+----------+----------+---------------+------+--------------------+-------------+-----------------+----------------+
|8W/rpnb48OMm47W2x...|    1| 139.6583505206638|36.561064568332455| 135.0| 66.0|247.0| 0.1701388888888889|              0.0625|  8.68154980903809|12.418971213537

### 8. PySpark: Sklearn Numerical Pipeline in PySpark

In [29]:
training_numerical_stages=feature_transformations.numerical_scaling(df=training_encoded)

### 9. PySpark: XGBoost Model

In [30]:
xgboost_model=create_pyspark_xgboost.xgboost_classifier(ml_df=training_encoded,
                                                        stages=training_numerical_stages,
                                                        model_storage_location=model_storage_location,
                                                        random_seed=random_seed)

You enabled use_gpu in spark local mode. Please make sure your local node has at least 4 GPUs


23/05/04 19:42:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[19:42:52] task 0 got new rank 0                                    (0 + 4) / 4]
[19:42:52] task 3 got new rank 1
[19:42:52] task 1 got new rank 2
[19:42:52] task 2 got new rank 3
/home/jovyan/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/xgboost/sklearn.py:782: UserWarning: Loading a native XGBoost model with Scikit-Learn interface.
  warnings.warn("Loading a native XGBoost model with Scikit-Learn interface.")


[19:42:55] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



You enabled use_gpu in spark local mode. Please make sure your local node has at least 4 GPUs


### 10. PySpark: Cross Validation

### 11. PySpark: Model Predictions

In [31]:
testing_predictions=model_predictions.create_predictions_with_model(test_df=testing_encoded, 
                                                                    model=xgboost_model)
testing_predictions.show(10)

+--------------------+-----+------+------------------+
|           PatientId|Chunk|target|        prediction|
+--------------------+-----+------+------------------+
|8W/rpnb48OMm47W2x...|    1|    49| 50.97040557861328|
|8W/rpnb48OMm47W2x...|    2|    -5|-5.244208812713623|
|CzndP9OQqEYW/LY7h...|    1|    77|  67.7061767578125|
|CzndP9OQqEYW/LY7h...|    2|    13| 10.86211109161377|
|ER+gxKFxHvXyfESG6...|    1|   -33|-39.08769226074219|
|ER+gxKFxHvXyfESG6...|    2|    55| 50.81527328491211|
|UNrE+DKLY8WEQsGhg...|    1|     8|0.8498030304908752|
|UNrE+DKLY8WEQsGhg...|    2|    27|25.882190704345703|
|WFanQktw0own7cwMg...|    1|   -22| -21.9498291015625|
|WFanQktw0own7cwMg...|    2|    21|19.890609741210938|
+--------------------+-----+------+------------------+
only showing top 10 rows



[19:43:04] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



### 12. PySpark: Model Evaluation

In [33]:
model_evaluation=evaluate_model.regression_evaluation(testing_predictions=testing_predictions, 
                                                          eval_csv_location=evaluation_metrics_output_storage)

[19:43:28] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[19:43:29] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[19:43:30] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, pl

In [34]:
model_evaluation.head()

,rmse,mse,r2,mae,var
0,4.488376,20.145517,0.97762,3.078903,937.275385


### 13. PySpark: XGBoost Classification Feature Importance

In [35]:
feature_importance_df=feature_importance.\
                        feature_importance_accuracy_gain(xgboost_model=xgboost_model, 
                                                         feature_importance_storage_location=feature_importance_storage_location)


In [36]:
feature_importance_df.head(10)

,Feature,Accuracy Gain
0,scaled_target,630.705566
1,scaled_Max,14.202221
2,scaled_TotalOutOfRange,3.835952
3,scaled_CountBelow,2.090200
4,scaled_StdFirstDiff,1.518541
5,scaled_Min,1.033857
6,scaled_AvgSecDiff,0.939486
7,scaled_StdDev,0.254810
8,scaled_CountAbove,0.096602
9,scaled_Mean,0.050581


### 14. PySpark: Feature Importance Plotting

In [ ]:
overall_feature_plot=xgboost_classification_plot.feature_overall_importance_plot(feature_importance_df=feature_importance_df,
                                                                                 overall_importance_plot_location=overall_feature_importance_plot_location)


In [ ]:
overall_feature_plot.show()

### 15.PySpark: Local Level Feature Importance --> Shap Pandas UDF

In [ ]:
# Add to reqs if this works
! pip install shap

In [ ]:
xgboost_model.stages[-1]

In [ ]:
import shap

In [ ]:
explainer = shap.TreeExplainer(xgboost_model.stages[-1])